In [1]:
#导包
from netCDF4 import Dataset
from tensorflow.keras import layers, models
from tensorflow.keras.layers import ConvLSTM2D,LSTM, BatchNormalization, LayerNormalization,Input, Conv3D, TimeDistributed, Flatten
from tensorflow.keras.layers import Concatenate,Conv2D,TimeDistributed, MaxPooling2D, Input, MaxPooling3D
from tensorflow.keras.layers import  Reshape,multiply
from tensorflow.keras.layers import Layer,Lambda,Dot,ReLU, Dense, Dropout, Activation, Flatten,Attention
from tensorflow.keras.utils import plot_model
from tensorflow.keras.models import Sequential,Model,load_model
from tensorflow.keras.optimizers import Adam,SGD
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from tensorflow.keras.losses import MeanSquaredError
from sklearn.metrics import r2_score,accuracy_score,precision_score
from tensorflow.keras.metrics import RootMeanSquaredError
from tensorflow.keras import regularizers
import tensorflow as tf
from datetime import datetime
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from tensorflow.keras import models, layers
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor
import tensorflow.keras as keras
from tensorflow.keras import layers, Sequential, regularizers
from tensorflow.keras.utils import  plot_model
# calculate RMSE
from sklearn.metrics import mean_squared_error
from math import sqrt
import pandas as pd 
import gc 
import gzip
import matplotlib.pyplot as plt
import numpy as np
import netCDF4 as nc
import match
import xarray as xr
import os 
%matplotlib inline

In [2]:
# 检查可用GPU数量
len(tf.config.experimental.list_physical_devices('GPU'))

1

# 读取数据

## SSH

In [3]:
data_ssh = nc.Dataset(r'D:/codeFile/jupyterDemo/3d_ts/BPNN/data/SSH/SSH_2005-2015_month.nc')
time_ssh = data_ssh['time'][:]
print('时间长度',len(time_ssh))

时间长度 132


In [4]:
# 确认经纬度，选取研究区域
ssh_lat = data_ssh['lat'][:].data   #0.125 -- 69.875
#print(ssh_lat[:])
ssh_lon = data_ssh['lon'][:].data   #110.125 -- 259.875
#print(ssh_lon[:])

In [5]:
ssh = data_ssh['ssh'][:].data
print(ssh.shape)
print(np.nanmax(ssh),np.nanmin(ssh))

(132, 280, 600)
2.20051 -0.43673334


## SSS

In [6]:
data_sss = nc.Dataset(r'D:/codeFile/jupyterDemo/3d_ts/BPNN/data/SSS/dataset-sss-ssd-rep-monthly_2005-2020.nc')

In [7]:
# 确定时间
time_sss = data_sss['time'][:132]
print(len(time_sss))

132


In [8]:
# 获取研究位置经纬度
sss_lat = data_sss['lat'][:].data   #0.125  -- 69.875 
#print(sss_lat[:])
sss_lon = data_sss['lon'][:].data   #110.125 -- 259.875
#print(sss_lon[:])

In [9]:
sss = data_sss['sos'][:132].data 
print(sss.shape)
sss = np.squeeze(sss)   # 移除大小为一的维度
print(sss.shape)

(132, 1, 280, 600)
(132, 280, 600)


In [10]:
print('最大值与最小值:',np.nanmax(sss),np.nanmin(sss))

最大值与最小值: 36.072178 13.604756


## SSW

In [11]:
data_ssw = nc.Dataset(r'D:/codeFile/jupyterDemo/3d_ts/BPNN/data/SSW/anon_ssw_2005_2018_Month_data.nc')
# 确定时间
time_ssw = data_ssw['time'][:132]
print(len(time_ssw))

132


In [12]:
lat_ssw = data_ssw['lat'][:].data   # lat 0.125    ----    69.875
#print(lat_ssw[314:594])
lon_ssw = data_ssw['lon'][:].data   # lon  110.125    ----    259.875
#print(lon_ssw[440:1040])

In [13]:
uwnd = data_ssw['uwnd'][:132,314:594,440:1040].data  
vwnd = data_ssw['vwnd'][:132,314:594,440:1040].data
print(uwnd.shape)
print(vwnd.shape)

(132, 280, 600)
(132, 280, 600)


In [14]:
np.nanmax(uwnd),np.nanmin(uwnd),np.nanmax(vwnd),np.nanmin(vwnd)

(12.433594, -11.7988205, 8.8359995, -13.645481)

## SST 海表面温度

In [15]:
data_3dt_1 = nc.Dataset(r'D:/codeFile/jupyterDemo/3d_ts/BPNN/data/3DT/dataset-armor-3d-rep-monthly_2005-2009.nc')
data_3dt_2 = nc.Dataset(r'D:/codeFile/jupyterDemo/3d_ts/BPNN/data/3DT/dataset-armor-3d-rep-monthly_2010-2015.nc')

In [16]:
sst_1 = data_3dt_1['to'][:].data
sst_2 = data_3dt_2['to'][:].data

In [17]:
print(sst_1.shape)
print(sst_2.shape)

(60, 43, 280, 600)
(72, 43, 280, 600)


In [18]:
temp_depth = np.append(sst_1,sst_2,axis=0)
print(temp_depth.shape)

(132, 43, 280, 600)


In [19]:
sst = temp_depth[:,0:1,:,:]

In [20]:
sst = np.squeeze(sst)
print(sst.shape)

(132, 280, 600)


## 3DS

In [21]:
data_3ds = nc.Dataset(r'D:/codeFile/jupyterDemo/3d_ts/BPNN/data/3DT/sali/3DS_end_2005_2015.nc')
sali_depth = data_3ds['so'][:].data
print(sali_depth.shape)

(132, 43, 280, 600)


In [22]:
depths = [0,5,10,15,20,25,30,35,40,45,50,55,60,65,70,80,90,100,125,150,175,200,225,250,275,300,350,400,
         450,500,550,600,700,800,900,1000,1100,1200,1300,1400,1500,1750,2000]
#depths_use = [50,100,125,150,200,250,300,400,500,600,700,800,900,1000]
#5-2000m
depths_use = [10,15,20,30,40,50,60,70,80,90,100,125,150,175,200,225,250,275,300,350,400,500,600,700,800,900,1000,1100,
              1300,1500,1750,2000]  # 长度为33

In [23]:
sali_depth_use = sali_depth[:,1:2]

In [24]:
for i,depth in enumerate(depths):
    if(depth in depths_use):
        sali_depth_use = np.concatenate((sali_depth_use,sali_depth[:,i:i+1]),axis=1)    

In [25]:
print(sali_depth_use.shape)
sali_depth_use = np.transpose(sali_depth_use,(0,2,3,1))
print(sali_depth_use.shape)

(132, 33, 280, 600)
(132, 280, 600, 33)


## 数据合并

In [26]:
print(ssh.shape,sst.shape,uwnd.shape,vwnd.shape,sss.shape)
cat_data =np.stack((ssh,sst,uwnd,vwnd,sss),axis=3)
cat_data.shape

(132, 280, 600) (132, 280, 600) (132, 280, 600) (132, 280, 600) (132, 280, 600)


(132, 280, 600, 5)

In [27]:
# 将表面数据和水下数据在最后一个维度拼接，将空值删除。
cat_surface_depth = np.concatenate((cat_data,sali_depth_use),axis=-1)
print(cat_surface_depth.shape)

(132, 280, 600, 38)


## 将两个温度异常区域的数据设置为nan

In [28]:
# test = temp_depth_use[:,:,:,14:]
# test[test < 0.4] = np.nan
# test[test > 10] = np.nan
# cat_data_ss_depth[:,:,:,19:] = test
# cat_data_ss_depth.shape 

## 去掉合并数据中的nan


In [29]:
cat_surface_depth.shape

(132, 280, 600, 38)

In [30]:
## 先对合并的数据进行reshape成（长*宽，时间，通道）
cat_surface_depth_reshape  = np.transpose(cat_surface_depth,(1,2,0,3))
print(cat_surface_depth_reshape.shape)
cat_surface_depth_reshape = cat_surface_depth_reshape.reshape ((-1,132*38))
print(cat_surface_depth_reshape.shape)

(280, 600, 132, 38)
(168000, 5016)


In [31]:
# 将填充值赋值为nan
cat_surface_depth_reshape[cat_surface_depth_reshape == 32767] = np.nan
print('cat_surface_depth_reshape删除nan值之前',cat_surface_depth_reshape.shape)
# 使用 numpy.isnan 函数检测NaN值
nan_mask = np.isnan(cat_surface_depth_reshape)
# 通过布尔索引删除NaN值
cat_surface_depth_nonan = cat_surface_depth_reshape[~nan_mask.any(axis=1)]   
# ~nan_mask.any(axis=1)只要第一个维度中含有False就会返回False
print('cat_surface_depth_reshape删除nan值之后',cat_surface_depth_nonan.shape)

cat_surface_depth_reshape删除nan值之前 (168000, 5016)
cat_surface_depth_reshape删除nan值之后 (104729, 5016)


In [32]:
cat_surface_depth_nonan = cat_surface_depth_nonan.reshape(-1,132,38)
print(cat_surface_depth_nonan.shape)
cat_surface_depth_nonan = np.transpose(cat_surface_depth_nonan,(1,0,2))
print(cat_surface_depth_nonan.shape)

(104729, 132, 38)
(132, 104729, 38)


## 将数据进行堆叠

In [33]:
cat_surface_depth_nonan.shape
surface_data = cat_surface_depth_nonan[:,:,:5]
sali_depth_use = cat_surface_depth_nonan[:,:,5:]
print(surface_data.shape,sali_depth_use.shape)

(132, 104729, 5) (132, 104729, 33)


In [34]:
available = len(surface_data) 
predict = 1

surface_data_shape = surface_data.shape[1:]
sali_depth_use_shape = sali_depth_use.shape[1:]

N_stack = 6

X_len = available - (N_stack-1)
Y_len = X_len

X = np.zeros(shape = (X_len, N_stack,surface_data_shape[0],surface_data_shape[1]))  
for i in range(X_len):
    X[i] = np.stack(surface_data[i:i+N_stack])
    #print(i)

Y = np.zeros(shape = (Y_len,predict,sali_depth_use_shape[0],sali_depth_use_shape[1]))
for i in range(Y_len):
    Y[i] = np.stack(sali_depth_use[i+N_stack-1 : i+N_stack-1+predict])
   # print(i+N_stack-1)

print(X.shape)
print(Y.shape)

(127, 6, 104729, 5)
(127, 1, 104729, 33)


# 验证集和测试集的划分

In [35]:
# 最后十二个月的数据作为test
test_num = 115
x_train = X[:test_num]
y_train = Y[:test_num]

x_test = X[test_num:]
y_test = Y[test_num:]
print(x_train.shape,y_train.shape,x_test.shape,y_test.shape )

(115, 6, 104729, 5) (115, 1, 104729, 33) (12, 6, 104729, 5) (12, 1, 104729, 33)


## 修改维度

In [36]:
x_train = np.transpose(x_train, (0,2,1,3))   #(个数，尺度，时间，通道)
x_train = x_train.reshape((-1,6,5))          #(个数*尺度，时间，通道)

y_train = np.transpose(y_train, (0,2,1,3))
y_train = y_train.reshape((-1,1,33))

x_test = np.transpose(x_test, (0,2,1,3))
x_test = x_test.reshape((-1,6,5))

y_test = np.transpose(y_test, (0,2,1,3))
y_test = y_test.reshape((-1,1,33))
print(x_train.shape,y_train.shape,x_test.shape,y_test.shape)

(12043835, 6, 5) (12043835, 1, 33) (1256748, 6, 5) (1256748, 1, 33)


## 划分训练数据和目标数据

In [37]:
print('x_test.shape:',x_test.shape)
print('y_test.shape',y_test.shape)
print('x_train',x_train.shape)
print('y_train',y_train.shape)

x_test.shape: (1256748, 6, 5)
y_test.shape (1256748, 1, 33)
x_train (12043835, 6, 5)
y_train (12043835, 1, 33)


In [38]:
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.3, random_state=100)
print(x_train.shape,x_test.shape,x_val.shape,y_train.shape,y_test.shape,y_val.shape)

(8430684, 6, 5) (1256748, 6, 5) (3613151, 6, 5) (8430684, 1, 33) (1256748, 1, 33) (3613151, 1, 33)


# 特征归一化

In [39]:
# 记录数据集的形状
x_train_shape = x_train.shape
y_train_shape = y_train.shape

x_test_shape = x_test.shape
y_test_shape = y_test.shape

x_val_shape = x_val.shape
y_val_shape = y_val.shape

In [40]:
scaler_f = StandardScaler()
x_train = scaler_f.fit_transform(x_train.reshape(-1,x_train_shape[2])) 
x_test = scaler_f.transform(x_test.reshape(-1,x_test_shape[2]))
x_val = scaler_f.transform(x_val.reshape(-1,x_val_shape[2]))

scaler_l = StandardScaler()
y_train = scaler_l.fit_transform(y_train.reshape(-1,y_train_shape[2])) 
y_test = scaler_l.transform(y_test.reshape(-1,y_test_shape[2])) 
y_val = scaler_l.transform(y_val.reshape(-1,y_val_shape[2])) 

In [41]:
x_train = np.reshape(x_train,(x_train_shape))
x_test = np.reshape(x_test,(x_test_shape))
x_val = np.reshape(x_val,(x_val_shape))
y_train = np.reshape(y_train,(y_train_shape[0],-1))
y_test = np.reshape(y_test,(y_test_shape[0],-1))
y_val = np.reshape(y_val,(y_val_shape[0],-1))

In [42]:
print(x_train.shape,x_test.shape,x_val.shape,y_train.shape,y_test.shape,y_val.shape)

(8430684, 6, 5) (1256748, 6, 5) (3613151, 6, 5) (8430684, 33) (1256748, 33) (3613151, 33)


# 定义网络结构

In [43]:
#  model2_input= Input(shape=(6,5))
#     x2 = MHA1(model2_input,model2_input)
#     x2 = LSTM(64, return_sequences=True)(x2)   
#     x2 = Dropout(0.2)(x2)
    
#     x2 = MHA2(x2,x2)
#     x2 = LSTM(32, return_sequences=True)(x2)
#     x2 = Dropout(0.2)(x2)
# #
#     x2 = MHA3(x2,x2)
#     x2 = Flatten()(x2)
#     out2 = Dense(1)(x2)

In [44]:
#  MHA2 = tf.keras.layers.MultiHeadAttention(num_heads=8, key_dim=8)
#     MHA3 = tf.keras.layers.MultiHeadAttention(num_heads=8, key_dim=4)
    
    
#     model2_input= Input(shape=(6,5))

#     x2 = LSTM(64, return_sequences=True)(model2_input)   
#     x2 = Dropout(0.2)(x2)
    
#     mha2 = MHA2(x2,x2)
#     x2 = tf.keras.layers.Add()([x2, mha2])
#     x2 = Activation('relu')(x2)
    
#     x2 = LSTM(32, return_sequences=True)(x2)
#     #x2 = Dropout(0.2)(x2)
# #
#     mha3 = MHA3(x2,x2)
#     x2 = tf.keras.layers.Add()([x2, mha3])
#     x2 = Activation('relu')(x2)
#     x2 = Flatten()(x2)
#     out2 = Dense(1)(x2)
    

In [45]:
# def my_model():
# #     MHA1 = tf.keras.layers.MultiHeadAttention(num_heads=2, key_dim=3)
#     MHA2 = tf.keras.layers.MultiHeadAttention(num_heads=4, key_dim=16)
# #     MHA3 = tf.keras.layers.MultiHeadAttention(num_heads=4, key_dim=8)
#     #MHA4 = tf.keras.layers.MultiHeadAttention(num_heads=8, key_dim=16)
    
#     model_input= Input(shape=(6,5))
    
# #     mha1 = MHA1(model_input,model_input)
# #     model = tf.keras.layers.Add()([model_input,mha1])
# #     model = tf.keras.layers.ReLU()(model)
    
#     model= LSTM(64, return_sequences=True)(model_input)   
#     model = Dropout(0.2)(model)
#     mha2 = MHA2(model,model)
#     model = tf.keras.layers.Add()([model,mha2])
#     model = tf.keras.layers.ReLU()(model)
    
#     model = LSTM(32, return_sequences=True)(model)
#     model = Dropout(0.2)(model)
# #     mha3 = MHA3(model,model)
# #     model = tf.keras.layers.Add()([model, mha3])
# #     model = tf.keras.layers.ReLU()(model)
    
# #     model_out = LSTM(32, return_sequences=True)(model)
# #     model = Dropout(0.3)(model)
# #     mha4 = MHA4(model,model)
# #     model = tf.keras.layers.Add()([model, mha4,model_out])
# #     model = tf.keras.layers.ReLU()(model)
    
    
#     model = Flatten()(model)
#     out = Dense(1)(model)
#     model = Model(inputs=model_input, outputs=out)
#     return model

In [46]:
# 注意力层的定制实现，以适应多次注意力调用
class Attention(Layer):
    def __init__(self, units=128, **kwargs):
        self.units = units
        super().__init__(**kwargs)
        
    def __call__(self, inputs):
        """
        Keras的多对一注意力机制。
        @param inputs: 3D tensor with shape (batch_size, time_steps, input_dim).
        @return: 2D tensor with shape (batch_size, 128)
        """
        
        hidden_states = inputs
        hidden_size = int(hidden_states.shape[2])  
        
        # Inside dense layer 内部全连接层
        #              hidden_states            dot               W            =>           score_first_part
        # (batch_size, time_steps, hidden_size) dot (hidden_size, hidden_size) => (batch_size, time_steps, hidden_size)
        # W是可训练权重矩阵 
        
        score_first_part = Dense(hidden_size, use_bias=False)(hidden_states) 
        # 进行一次全连接，本质上没有太大意义，只是为了增加一下模型复杂度，
        # 并且改变一下前T个时间步的特征表示,这里其实也可以不进行全连接，直接使用hidden_states
        
        #            score_first_part           dot        last_hidden_state     => attention_weights
        # (batch_size, time_steps, hidden_size) dot   (batch_size, hidden_size)  => (batch_size, time_steps)
        # # 注意力，增加对最后一个时间步长的关注
        
        h_t = Lambda(lambda x: x[:, -1, :], output_shape=(hidden_size,))(hidden_states)  # 取出最后一个时间步的特征
        score = Dot(axes=[1, 2])([h_t, score_first_part])  # 将最后一个时间步长与前T个时间步长进行点积
        attention_weights = Activation('softmax')(score)   # 求权重（将权值通过softmax进行归一化，得到直接可用的权重）
        
        
        # (batch_size, time_steps, hidden_size) dot (batch_size, time_steps) => (batch_size, hidden_size)
        context_vector = Dot(axes=[1, 1])([hidden_states, attention_weights])  # 上下文向量
        pre_activation = Concatenate()([context_vector, h_t]) #再次将最后一个时间步的信息加入，本质上是一个残差连接，这步可以省略
        attention_vector = Dense(self.units, use_bias=False, activation='tanh')(pre_activation) # 用一个全连接层进行合并
        
        print('注意力被执行了一次')
        return attention_vector

    def get_config(self):
        return {'units': self.units}

    @classmethod
    def from_config(cls, config):
        return cls(**config)

In [47]:
def my_model():
    
    model_input= Input(shape=(6,5))
   
    model= LSTM(64, return_sequences=True)(model_input)   
    model = Dropout(0.2)(model_input)
    
    model = LSTM(32, return_sequences=True)(model)
    model = Attention()(model)
    
    model = Flatten()(model)
    
    out = Dense(1)(model)
    model = Model(inputs=model_input, outputs=out)
    return model

In [48]:
# import graphviz
# model = my_model()
# #model.summary()
# plot_model(model,show_shapes=True,show_layer_names=True)

## 对指定层进行训练

In [ ]:
num_depths = 1
num_features = 5
# # 编译模型
model.compile(optimizer='adam', loss='mean_squared_error')
checkpoint_path='./LSTM_MHA_8.h5'
keras_callbacks   = [
      EarlyStopping(monitor='val_loss', patience=30, mode='min', min_delta=0.001),
      ModelCheckpoint(checkpoint_path, monitor='val_loss', save_best_only=True, mode='min')
]

In [ ]:
num_epochs = 1000
batch_size = 10240

history = model.fit(x_train, y_train[:,14:15], validation_data=(x_val,y_val[:,14:15]),
                    epochs=num_epochs, batch_size=batch_size, verbose=2, callbacks=keras_callbacks)

## 网格搜索交叉验证寻找最佳参数

In [ ]:
# model = KerasRegressor(build_fn=create_model,verbose=2)

In [ ]:
# param_grid = {
#     'neurons': [15,30,50,100,150,200],
# }

In [ ]:
# # 使用GridSearchCV进行交叉验证并搜索最佳参数组合：
# grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5)
# grid_result = grid_search.fit(x_train, y_train[:,14:15],epochs=200,verbose=2,batch_size=1024)

In [ ]:
# print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

## 训练30 - 1000m

In [67]:
def train_model(x_train,y_train,x_val,y_val,name):
    # 创建网络
    model = my_model()
    # 编译网络
    model.compile(loss='mse', optimizer='adam')
    checkpoint_path=name
    keras_callbacks   = [
          EarlyStopping(monitor='val_loss', patience=8, mode='min', min_delta=0.0001),
          ModelCheckpoint(checkpoint_path, monitor='val_loss', save_best_only=True, mode='min')
    ]
    model.fit(x_train, y_train, validation_data=(x_val,y_val),
                    epochs=200, batch_size=10240, verbose=2, callbacks=keras_callbacks)

In [68]:
depth = [5,10,15,20,30,40,50,60,70,80,90,100,125,150,175,200,225,250,275,300,350,400,500,600,700,800,900,1000,1100,
              1300,1500,1750,2000] 

In [69]:
print(x_train.shape,x_test.shape,x_val.shape,y_train.shape,y_test.shape,y_val.shape)

(8430684, 6, 5) (1256748, 6, 5) (3613151, 6, 5) (8430684, 33) (1256748, 33) (3613151, 33)


In [70]:
# 训练5 - 2000m
for i in range(0,33):
    train_model(x_train,y_train[:,i:i+1],x_val,y_val[:,i:i+1],'../model/LSTM/sali/LSTM_sali_'+str(depth[i])+'m.h5')

注意力被执行了一次
Epoch 1/200
824/824 - 8s - loss: 0.0263 - val_loss: 0.0025
Epoch 2/200
824/824 - 6s - loss: 0.0038 - val_loss: 0.0025
Epoch 3/200
824/824 - 6s - loss: 0.0034 - val_loss: 0.0024
Epoch 4/200
824/824 - 6s - loss: 0.0032 - val_loss: 0.0024
Epoch 5/200
824/824 - 6s - loss: 0.0031 - val_loss: 0.0024
Epoch 6/200
824/824 - 6s - loss: 0.0031 - val_loss: 0.0024
Epoch 7/200
824/824 - 6s - loss: 0.0030 - val_loss: 0.0024
Epoch 8/200
824/824 - 6s - loss: 0.0030 - val_loss: 0.0024
Epoch 9/200
824/824 - 6s - loss: 0.0029 - val_loss: 0.0026
Epoch 10/200
824/824 - 6s - loss: 0.0029 - val_loss: 0.0025
Epoch 11/200
824/824 - 6s - loss: 0.0028 - val_loss: 0.0025
注意力被执行了一次
Epoch 1/200
824/824 - 9s - loss: 0.0296 - val_loss: 0.0069
Epoch 2/200
824/824 - 6s - loss: 0.0081 - val_loss: 0.0067
Epoch 3/200
824/824 - 6s - loss: 0.0078 - val_loss: 0.0067
Epoch 4/200
824/824 - 6s - loss: 0.0076 - val_loss: 0.0067
Epoch 5/200
824/824 - 6s - loss: 0.0074 - val_loss: 0.0066
Epoch 6/200
824/824 - 6s - loss: 0

Epoch 4/200
824/824 - 6s - loss: 0.0137 - val_loss: 0.0122
Epoch 5/200
824/824 - 6s - loss: 0.0132 - val_loss: 0.0119
Epoch 6/200
824/824 - 6s - loss: 0.0128 - val_loss: 0.0119
Epoch 7/200
824/824 - 6s - loss: 0.0125 - val_loss: 0.0115
Epoch 8/200
824/824 - 6s - loss: 0.0122 - val_loss: 0.0111
Epoch 9/200
824/824 - 6s - loss: 0.0120 - val_loss: 0.0109
Epoch 10/200
824/824 - 6s - loss: 0.0118 - val_loss: 0.0108
Epoch 11/200
824/824 - 6s - loss: 0.0116 - val_loss: 0.0106
Epoch 12/200
824/824 - 6s - loss: 0.0114 - val_loss: 0.0106
Epoch 13/200
824/824 - 6s - loss: 0.0112 - val_loss: 0.0104
Epoch 14/200
824/824 - 6s - loss: 0.0111 - val_loss: 0.0102
Epoch 15/200
824/824 - 6s - loss: 0.0110 - val_loss: 0.0103
Epoch 16/200
824/824 - 6s - loss: 0.0109 - val_loss: 0.0101
Epoch 17/200
824/824 - 6s - loss: 0.0107 - val_loss: 0.0100
Epoch 18/200
824/824 - 6s - loss: 0.0106 - val_loss: 0.0098
Epoch 19/200
824/824 - 6s - loss: 0.0105 - val_loss: 0.0097
Epoch 20/200
824/824 - 6s - loss: 0.0104 - val

Epoch 55/200
824/824 - 6s - loss: 0.0094 - val_loss: 0.0084
Epoch 56/200
824/824 - 6s - loss: 0.0093 - val_loss: 0.0085
Epoch 57/200
824/824 - 6s - loss: 0.0093 - val_loss: 0.0084
Epoch 58/200
824/824 - 6s - loss: 0.0093 - val_loss: 0.0083
Epoch 59/200
824/824 - 6s - loss: 0.0092 - val_loss: 0.0083
Epoch 60/200
824/824 - 6s - loss: 0.0092 - val_loss: 0.0082
Epoch 61/200
824/824 - 6s - loss: 0.0092 - val_loss: 0.0082
Epoch 62/200
824/824 - 6s - loss: 0.0091 - val_loss: 0.0082
Epoch 63/200
824/824 - 6s - loss: 0.0091 - val_loss: 0.0081
Epoch 64/200
824/824 - 6s - loss: 0.0091 - val_loss: 0.0081
Epoch 65/200
824/824 - 6s - loss: 0.0091 - val_loss: 0.0081
Epoch 66/200
824/824 - 6s - loss: 0.0090 - val_loss: 0.0081
Epoch 67/200
824/824 - 6s - loss: 0.0090 - val_loss: 0.0080
Epoch 68/200
824/824 - 6s - loss: 0.0090 - val_loss: 0.0081
Epoch 69/200
824/824 - 6s - loss: 0.0090 - val_loss: 0.0081
Epoch 70/200
824/824 - 6s - loss: 0.0090 - val_loss: 0.0080
Epoch 71/200
824/824 - 6s - loss: 0.0089

Epoch 13/200
824/824 - 6s - loss: 0.0135 - val_loss: 0.0127
Epoch 14/200
824/824 - 6s - loss: 0.0133 - val_loss: 0.0123
Epoch 15/200
824/824 - 6s - loss: 0.0132 - val_loss: 0.0122
Epoch 16/200
824/824 - 6s - loss: 0.0130 - val_loss: 0.0121
Epoch 17/200
824/824 - 6s - loss: 0.0128 - val_loss: 0.0119
Epoch 18/200
824/824 - 6s - loss: 0.0127 - val_loss: 0.0119
Epoch 19/200
824/824 - 6s - loss: 0.0126 - val_loss: 0.0116
Epoch 20/200
824/824 - 6s - loss: 0.0124 - val_loss: 0.0116
Epoch 21/200
824/824 - 6s - loss: 0.0123 - val_loss: 0.0114
Epoch 22/200
824/824 - 6s - loss: 0.0122 - val_loss: 0.0114
Epoch 23/200
824/824 - 6s - loss: 0.0121 - val_loss: 0.0111
Epoch 24/200
824/824 - 6s - loss: 0.0119 - val_loss: 0.0110
Epoch 25/200
824/824 - 6s - loss: 0.0118 - val_loss: 0.0109
Epoch 26/200
824/824 - 6s - loss: 0.0117 - val_loss: 0.0107
Epoch 27/200
824/824 - 6s - loss: 0.0116 - val_loss: 0.0107
Epoch 28/200
824/824 - 6s - loss: 0.0116 - val_loss: 0.0106
Epoch 29/200
824/824 - 6s - loss: 0.0115

Epoch 53/200
824/824 - 6s - loss: 0.0105 - val_loss: 0.0094
Epoch 54/200
824/824 - 6s - loss: 0.0104 - val_loss: 0.0094
Epoch 55/200
824/824 - 6s - loss: 0.0104 - val_loss: 0.0093
Epoch 56/200
824/824 - 6s - loss: 0.0104 - val_loss: 0.0093
Epoch 57/200
824/824 - 6s - loss: 0.0103 - val_loss: 0.0094
Epoch 58/200
824/824 - 6s - loss: 0.0103 - val_loss: 0.0092
Epoch 59/200
824/824 - 6s - loss: 0.0103 - val_loss: 0.0091
Epoch 60/200
824/824 - 6s - loss: 0.0102 - val_loss: 0.0091
Epoch 61/200
824/824 - 6s - loss: 0.0102 - val_loss: 0.0091
Epoch 62/200
824/824 - 6s - loss: 0.0102 - val_loss: 0.0090
Epoch 63/200
824/824 - 6s - loss: 0.0102 - val_loss: 0.0091
Epoch 64/200
824/824 - 6s - loss: 0.0101 - val_loss: 0.0090
Epoch 65/200
824/824 - 6s - loss: 0.0101 - val_loss: 0.0089
Epoch 66/200
824/824 - 6s - loss: 0.0101 - val_loss: 0.0089
Epoch 67/200
824/824 - 6s - loss: 0.0101 - val_loss: 0.0089
Epoch 68/200
824/824 - 6s - loss: 0.0100 - val_loss: 0.0089
Epoch 69/200
824/824 - 6s - loss: 0.0100

...

Epoch 105/200
824/824 - 6s - loss: 0.0101 - val_loss: 0.0086
Epoch 106/200
824/824 - 6s - loss: 0.0101 - val_loss: 0.0086
Epoch 107/200
824/824 - 6s - loss: 0.0101 - val_loss: 0.0087
Epoch 108/200
824/824 - 6s - loss: 0.0101 - val_loss: 0.0086
Epoch 109/200
824/824 - 6s - loss: 0.0101 - val_loss: 0.0086
Epoch 110/200
824/824 - 6s - loss: 0.0101 - val_loss: 0.0085
Epoch 111/200
824/824 - 6s - loss: 0.0101 - val_loss: 0.0085
Epoch 112/200
824/824 - 6s - loss: 0.0100 - val_loss: 0.0084
Epoch 113/200
824/824 - 6s - loss: 0.0100 - val_loss: 0.0085
Epoch 114/200
824/824 - 6s - loss: 0.0100 - val_loss: 0.0085
Epoch 115/200
824/824 - 6s - loss: 0.0100 - val_loss: 0.0086
Epoch 116/200
824/824 - 6s - loss: 0.0100 - val_loss: 0.0084
Epoch 117/200
824/824 - 6s - loss: 0.0099 - val_loss: 0.0086
Epoch 118/200
824/824 - 6s - loss: 0.0099 - val_loss: 0.0084
注意力被执行了一次
Epoch 1/200
824/824 - 9s - loss: 0.0952 - val_loss: 0.0382
Epoch 2/200
824/824 - 6s - loss: 0.0374 - val_loss: 0.0304
Epoch 3/200
824/82

Epoch 29/200
824/824 - 6s - loss: 0.0167 - val_loss: 0.0147
Epoch 30/200
824/824 - 6s - loss: 0.0165 - val_loss: 0.0148
Epoch 31/200
824/824 - 6s - loss: 0.0164 - val_loss: 0.0144
Epoch 32/200
824/824 - 6s - loss: 0.0162 - val_loss: 0.0143
Epoch 33/200
824/824 - 6s - loss: 0.0160 - val_loss: 0.0148
Epoch 34/200
824/824 - 6s - loss: 0.0159 - val_loss: 0.0140
Epoch 35/200
824/824 - 6s - loss: 0.0157 - val_loss: 0.0140
Epoch 36/200
824/824 - 6s - loss: 0.0156 - val_loss: 0.0137
Epoch 37/200
824/824 - 6s - loss: 0.0155 - val_loss: 0.0136
Epoch 38/200
824/824 - 6s - loss: 0.0153 - val_loss: 0.0137
Epoch 39/200
824/824 - 6s - loss: 0.0152 - val_loss: 0.0132
Epoch 40/200
824/824 - 6s - loss: 0.0151 - val_loss: 0.0133
Epoch 41/200
824/824 - 6s - loss: 0.0150 - val_loss: 0.0131
Epoch 42/200
824/824 - 6s - loss: 0.0148 - val_loss: 0.0134
Epoch 43/200
824/824 - 6s - loss: 0.0148 - val_loss: 0.0131
Epoch 44/200
824/824 - 6s - loss: 0.0146 - val_loss: 0.0128
Epoch 45/200
824/824 - 6s - loss: 0.0146

824/824 - 6s - loss: 0.0161 - val_loss: 0.0141
Epoch 41/200
824/824 - 6s - loss: 0.0160 - val_loss: 0.0140
Epoch 42/200
824/824 - 6s - loss: 0.0157 - val_loss: 0.0137
Epoch 43/200
824/824 - 6s - loss: 0.0158 - val_loss: 0.0136
Epoch 44/200
824/824 - 6s - loss: 0.0154 - val_loss: 0.0138
Epoch 45/200
824/824 - 6s - loss: 0.0154 - val_loss: 0.0137
Epoch 46/200
824/824 - 6s - loss: 0.0156 - val_loss: 0.0134
Epoch 47/200
824/824 - 6s - loss: 0.0151 - val_loss: 0.0136
Epoch 48/200
824/824 - 6s - loss: 0.0150 - val_loss: 0.0131
Epoch 49/200
824/824 - 6s - loss: 0.0150 - val_loss: 0.0132
Epoch 50/200
824/824 - 6s - loss: 0.0148 - val_loss: 0.0130
Epoch 51/200
824/824 - 6s - loss: 0.0148 - val_loss: 0.0130
Epoch 52/200
824/824 - 6s - loss: 0.0147 - val_loss: 0.0127
Epoch 53/200
824/824 - 6s - loss: 0.0145 - val_loss: 0.0126
Epoch 54/200
824/824 - 6s - loss: 0.0145 - val_loss: 0.0133
Epoch 55/200
824/824 - 6s - loss: 0.0145 - val_loss: 0.0125
Epoch 56/200
824/824 - 6s - loss: 0.0143 - val_loss: 

Epoch 62/200
824/824 - 6s - loss: 0.0144 - val_loss: 0.0127
Epoch 63/200
824/824 - 6s - loss: 0.0143 - val_loss: 0.0125
Epoch 64/200
824/824 - 6s - loss: 0.0142 - val_loss: 0.0124
Epoch 65/200
824/824 - 6s - loss: 0.0143 - val_loss: 0.0128
Epoch 66/200
824/824 - 6s - loss: 0.0143 - val_loss: 0.0122
Epoch 67/200
824/824 - 6s - loss: 0.0140 - val_loss: 0.0122
Epoch 68/200
824/824 - 6s - loss: 0.0141 - val_loss: 0.0122
Epoch 69/200
824/824 - 6s - loss: 0.0138 - val_loss: 0.0125
Epoch 70/200
824/824 - 6s - loss: 0.0139 - val_loss: 0.0121
Epoch 71/200
824/824 - 6s - loss: 0.0138 - val_loss: 0.0122
Epoch 72/200
824/824 - 6s - loss: 0.0137 - val_loss: 0.0120
Epoch 73/200
824/824 - 6s - loss: 0.0137 - val_loss: 0.0120
Epoch 74/200
824/824 - 6s - loss: 0.0138 - val_loss: 0.0119
Epoch 75/200
824/824 - 6s - loss: 0.0135 - val_loss: 0.0124
Epoch 76/200
824/824 - 6s - loss: 0.0136 - val_loss: 0.0118
Epoch 77/200
824/824 - 6s - loss: 0.0134 - val_loss: 0.0119
Epoch 78/200
824/824 - 6s - loss: 0.0134

Epoch 63/200
824/824 - 6s - loss: 0.0073 - val_loss: 0.0068
Epoch 64/200
824/824 - 6s - loss: 0.0069 - val_loss: 0.0059
Epoch 65/200
824/824 - 6s - loss: 0.0069 - val_loss: 0.0063
Epoch 66/200
824/824 - 6s - loss: 0.0067 - val_loss: 0.0058
Epoch 67/200
824/824 - 6s - loss: 0.0071 - val_loss: 0.0059
Epoch 68/200
824/824 - 6s - loss: 0.0067 - val_loss: 0.0060
Epoch 69/200
824/824 - 6s - loss: 0.0066 - val_loss: 0.0060
Epoch 70/200
824/824 - 6s - loss: 0.0067 - val_loss: 0.0056
Epoch 71/200
824/824 - 6s - loss: 0.0065 - val_loss: 0.0056
Epoch 72/200
824/824 - 6s - loss: 0.0068 - val_loss: 0.0056
Epoch 73/200
824/824 - 6s - loss: 0.0066 - val_loss: 0.0059
Epoch 74/200
824/824 - 6s - loss: 0.0065 - val_loss: 0.0056
Epoch 75/200
824/824 - 6s - loss: 0.0064 - val_loss: 0.0055
Epoch 76/200
824/824 - 6s - loss: 0.0063 - val_loss: 0.0055
Epoch 77/200
824/824 - 6s - loss: 0.0064 - val_loss: 0.0059
Epoch 78/200
824/824 - 6s - loss: 0.0064 - val_loss: 0.0053
Epoch 79/200
824/824 - 6s - loss: 0.0062

## 模型评估

In [72]:
# 评估函数定义
# 异常相关系数
import math
def acc(actual, predicted):
    pred_avg = np.average(predicted)

    act_avg = np.average(actual)
    diff_pred = predicted - pred_avg
    diff_act = actual - act_avg
    numerator = np.mean(np.sum(diff_pred*diff_act, axis=0))
    denominator = math.sqrt(np.mean(np.sum(diff_pred**2, axis=0)) * np.mean(np.sum(diff_act**2, axis=0)))
    ret_val = numerator/denominator
    return (100 * ret_val)


# 查看损失
def look_loss(history):
    plt.figure(figsize=(6.3, 2.5), dpi = 100)
    plt.plot(history.history['loss'], label='training data')
    plt.plot(history.history['val_loss'], label='validation data')
    plt.title('Loss')
    plt.ylabel('MSE Loss')
    plt.xlabel('Number of epochs')
    plt.grid()
    plt.legend(loc="upper right")
    plt.show()


# 模型预测
def all_estimate(best_model,x_test,y_test):
    testPred_1 = best_model.predict(x_test)
    a = y_test
    y_test_p = a.reshape(-1,1)
    testPred_p = testPred_1.reshape(-1,1)


    rmse = sqrt(mean_squared_error(y_test_p,testPred_p))
    print('Test RMSE: %.3f' % rmse)

    print('acc:',acc(y_test_p,testPred_p))

    r2 = r2_score(y_test_p,testPred_p)
    print("R² score:", r2)  
    

def respective_estimate(best_model,x_test,y_test):
    re2_list = []
    rmse_list = []
    for i in range(6):
        # 模型预测
        testPred = best_model.predict(x_test[i:i+1])
        a = y_test[i:i+1]
        y_test_p = a.reshape(-1,1)
        testPred_p = testPred.reshape(-1,1)
        r2 = r2_score(y_test_p,testPred_p)
        rmse = sqrt(mean_squared_error(y_test_p,testPred_p))
        re2_list.append(r2)
        rmse_list.append(rmse)    
    return re2_list,rmse_list

### 对指定层进行评估

In [ ]:
best_model = tf.keras.models.load_model('LSTM_MHA_8.h5')
all_estimate(best_model,x_test,y_test[:,14:15])

### 对所有层进行评估

In [73]:
# 模型预测
for i in range(0,33):
    best_model = tf.keras.models.load_model('D:/codeFile/jupyterDemo/3d_ts/BPNN/model/LSTM/sali/LSTM_sali_'+str(depth[i])+'m.h5')
    print(str(depth[i])+'测试集：')
    all_estimate(best_model,x_test,y_test[:,i:i+1])
# re2_list,rmse_list = respective_estimate(best_model,x_test,y_test[:,:,:,:,0:5])
# re2_list,rmse_list

5测试集：
Test RMSE: 0.053
acc: 99.86097639687095
R² score: 0.9971981847433738
10测试集：
Test RMSE: 0.080
acc: 99.68526391117692
R² score: 0.9936983417751639
15测试集：
Test RMSE: 0.084
acc: 99.6615534471325
R² score: 0.9931128563125754
20测试集：
Test RMSE: 0.092
acc: 99.58727423076888
R² score: 0.9917221368877246
30测试集：
Test RMSE: 0.106
acc: 99.45622810450217
R² score: 0.9889039470220572
40测试集：
Test RMSE: 0.130
acc: 99.15482881356365
R² score: 0.983127385071642
50测试集：
Test RMSE: 0.144
acc: 98.98253563834423
R² score: 0.9797402287213278
60测试集：
Test RMSE: 0.155
acc: 98.8397729361961
R² score: 0.9768624570285477
70测试集：
Test RMSE: 0.161
acc: 98.74478657068387
R² score: 0.9750333367508117
80测试集：
Test RMSE: 0.166
acc: 98.66054161495687
R² score: 0.9733042103456326
90测试集：
Test RMSE: 0.170
acc: 98.57917104201496
R² score: 0.9716859352598562
100测试集：
Test RMSE: 0.179
acc: 98.41892878487425
R² score: 0.9680507064398273
125测试集：
Test RMSE: 0.198
acc: 98.0258230083591
R² score: 0.960007641910558
150测试集：
Test RMS

# 保存预测结果

In [74]:
# 模型预测
result = [] 
for i in range(0,33):
    best_model = tf.keras.models.load_model('D:/codeFile/jupyterDemo/3d_ts/BPNN/model/LSTM/sali/LSTM_sali_'+str(depth[i])+'m.h5')
    #print('预测第'+str(depth[i]+'m的温度')
    testPred = best_model.predict(x_test)
    result.append(testPred)

In [79]:
len(result),len(result[0]),len(result[0][0])

(33, 1256748, 1)

In [80]:
result = np.array(result)
print(result.shape)
result = np.transpose(result,(1,2,0))
print(result.shape)
result = result.reshape(-1,33)
print(result.shape)

(33, 1256748, 1)
(1256748, 1, 33)
(1256748, 33)


## 对预测结果进行反归一化

In [82]:
result.shape,y_test.shape 

((1256748, 33), (1256748, 33))

In [83]:
result_unscaled = scaler_l.inverse_transform(result)
y_test_unscaled = scaler_l.inverse_transform(y_test)

## 评估反归一化之后的结果

In [89]:
for i in range(0,33):
    r2 = r2_score(y_test_unscaled[:,i],result_unscaled[:,i])
    #print(str(depth[i])+'m R² score:',r2)
    print('%.4f' % r2)

0.9972
0.9937
0.9931
0.9917
0.9889
0.9831
0.9797
0.9769
0.9750
0.9733
0.9717
0.9681
0.9600
0.9553
0.9464
0.9414
0.9431
0.9497
0.9562
0.9530
0.9554
0.9569
0.9607
0.9665
0.9651
0.9626
0.9586
0.9587
0.9613
0.9540
0.9366
0.9304
0.9014


In [87]:
for i in range(0,33):
#     mse =  mean_squared_error(y_test_unscaled[:,i],result_unscaled[:,i])
#     print(str(depth[i])+'m Test mse: %.3f' % mse)
    rmse = sqrt(mean_squared_error(y_test_unscaled[:,i],result_unscaled[:,i]))
    print('%.3f' % rmse)

0.045
0.067
0.070
0.077
0.088
0.107
0.116
0.124
0.128
0.131
0.131
0.135
0.133
0.124
0.119
0.110
0.098
0.085
0.074
0.073
0.064
0.058
0.047
0.038
0.034
0.030
0.026
0.022
0.018
0.016
0.016
0.015
0.018


# 预测结果可视化

## 为归一化之后的预测值和真实值填充nan

In [90]:
cat_surface_depth.shape 

(132, 280, 600, 38)

In [91]:
cat_surface_depth_test = np.transpose(cat_surface_depth,(1,2,0,3))
print(cat_surface_depth_test.shape)
cat_surface_depth_test = cat_surface_depth_test.reshape ((-1,132*38))
print(cat_surface_depth_test.shape)

(280, 600, 132, 38)
(168000, 5016)


In [95]:
cat_surface_depth_test[cat_surface_depth_test == 32767] = np.nan
print('cat_surface_depth_test删除nan值之前',cat_surface_depth_test.shape)
# 使用 numpy.isnan 函数检测NaN值
nan_mask = np.isnan(cat_surface_depth_test)

cat_surface_depth_test删除nan值之前 (168000, 5016)


In [96]:
nan_mask[nan_mask.any(axis=1) == True] = True   # nan为True,只要有一个通道存在nan,就都设置为nan
nan_masks = nan_mask[:,1]    #得到图像的masks

In [98]:
nan_masks.shape 

(168000,)

In [99]:
### 创建一个形状与nan_masks相同且全是nan的数组
testPred_with_nan = np.full((168000,12*33), np.nan)  
print(testPred_with_nan.shape)

y_test_with_nan = np.full((168000,12*33), np.nan)  
print(y_test_with_nan.shape)

(168000, 396)
(168000, 396)


In [100]:
result_unscaled_reshape = result_unscaled.reshape(12,-1,33)
print(result_unscaled_reshape.shape)
result_unscaled_reshape = np.transpose(result_unscaled_reshape,(1,0,2))
print(result_unscaled_reshape.shape)
result_unscaled_reshape = result_unscaled_reshape.reshape(-1,12*33)
print(result_unscaled_reshape.shape)

# ======================================== #
y_test_unscaled_reshape = y_test_unscaled.reshape(12,-1,33)
print(y_test_unscaled_reshape.shape)
y_test_unscaled_reshape = np.transpose(y_test_unscaled_reshape,(1,0,2))
print(y_test_unscaled_reshape.shape)
y_test_unscaled_reshape = y_test_unscaled_reshape.reshape(-1,12*33)
print(y_test_unscaled_reshape.shape)


(12, 104729, 33)
(104729, 12, 33)
(104729, 396)
(12, 104729, 33)
(104729, 12, 33)
(104729, 396)


In [102]:
y_test_unscaled.shape 

(1256748, 33)

In [103]:
num = 0
for i,nan_mask in enumerate(nan_masks):   # i从0开始

    if (nan_mask == False): # False 代表没有nan，使用预测值进行填充
        testPred_with_nan[i] = result_unscaled_reshape[num]
        y_test_with_nan[i] = y_test_unscaled_reshape[num]
        num = num + 1
print(num)

104729


In [104]:
testPred_with_nan = testPred_with_nan.reshape(280, 600,12,33)
testPred_with_nan = np.transpose(testPred_with_nan,(2,3,0,1))


y_test_with_nan = y_test_with_nan.reshape(280, 600,12,33)
y_test_with_nan = np.transpose(y_test_with_nan,(2,3,0,1))

print(testPred_with_nan.shape,y_test_with_nan.shape)

(12, 33, 280, 600) (12, 33, 280, 600)


# 结果可视化

In [105]:
lat = []
lon = []
time = []

In [106]:
lon.append(data_ssh['lon'][:].data)
lat.append(data_ssh['lat'][:].data)

In [107]:
start_year = 2015
end_year =2015
time = []
date_range = pd.date_range(datetime(start_year,1,1),datetime(end_year+1,1,1),freq='1M')
len(date_range)
for i in range(len(date_range)):
    time.append(i)

In [ ]:
new_NC.close()

In [109]:
new_NC = nc.Dataset("D:/codeFile/jupyterDemo/3d_ts/BPNN/model/LSTM/result/result5_2000m_sali_LSTM.nc", 'w', format='NETCDF4')

new_NC.createDimension('lat', len(lat[0]))
new_NC.createDimension('lon', len(lon[0]))
new_NC.createDimension('depth', 33)
new_NC.createDimension('time', len(time))

new_NC.createVariable('lat', 'f', ("lat"))
new_NC.createVariable('lon', 'f', ("lon"))
new_NC.createVariable('depth', 'f', ("depth"))
new_NC.createVariable('true_ss', 'f',("time","depth","lat","lon"))
new_NC.createVariable('pred_ss', 'f',("time","depth","lat","lon"))

time_var = new_NC.createVariable('time', 'f4',("time"))
time_var.units = 'months since 2015-01-15'
time_var.long_name = 'Months in Monthly Means'
time_var.axis = 'T'

#向变量中填充数据
new_NC.variables['lat'][:] = lat[0]
new_NC.variables['lon'][:] = lon[0]
new_NC.variables['time'][:] = np.array(time)
new_NC.variables['depth'][:] = depth

new_NC.variables['true_ss'][:]=np.array(y_test_with_nan)
new_NC.variables['pred_ss'][:]=np.array(testPred_with_nan)


#最后记得关闭文件
new_NC.close()
